In [23]:
import tempfile
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_openai.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from dotenv import load_dotenv

load_dotenv()

MODEL = "gpt-3.5-turbo"

def importa_documento(files):
    if files is not None:
        paginas = []
        for file in files:
            with tempfile.NamedTemporaryFile(delete=False, suffix=".pdf") as temp_file:
                temp_file.write(file.read())
                temp_file_path = temp_file.name

            loader = PyPDFLoader(temp_file_path)
            paginas.extend(loader.load())
        return paginas
    else:
        return []
    
def split_documents(documents):
    chunk_size = 2500
    chunk_overlap = 250
    separators = ['\n\n', '\n', '.', '?', '!', ' ', '']
    char_split = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=separators,
    )
    documents = char_split.split_documents(
    documents
    )
    return documents

def cria_vector_store(documents):
    embbedding_model = OpenAIEmbeddings()
    vector_store = FAISS.from_documents(documents, embbedding_model)
    return vector_store

def cria_chain_chat(vector_store):
    chat = ChatOpenAI(model=MODEL)
    memory = ConversationBufferMemory(return_messages=True,
                                      memory_key="chat_history",
                                      output_key='answer')
    chat_chain = ConversationalRetrievalChain.from_llm(
        llm = chat,
        memory = memory,
        return_source_documents = True,
        retriever = vector_store.as_retriever(),
        verbose=True
    )
    return chat_chain


In [24]:
doc = ('/home/carlosaraujo/Downloads/relatorio-consolidado-anual-2023.pdf')
doc = open(doc, 'rb')
docs = importa_documento([doc])
docs = split_documents(docs)
vector_store = cria_vector_store(docs)
chat_chain = cria_chain_chat(vector_store)

In [25]:
chat_chain.invoke({'question': 'resuma o documento em anexo!'})



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the user's question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Reembolsos de empréstimos de ativos
Reembolsos de empréstimos de ativos - 2023
Você não possui dados disponíveis no período selecionado.
Relatório anual consolidado
As informações disponíveis na Área do Investidor e seus relatórios são para fins consultivos. Por isso, você, investidor que fará a sua declaração de imposto de renda, precisa solicitar junto a sua
corretora ou custodiante os informes de rendimentos emitidos nos termos da regulamentação da Receita Federal.
acesse investidor.B3.com.br 8/8

Total
R$ 4.489,05
Relatório anual consolidado
As informações disponíveis na Área do Investidor e seus relatórios são para fins consultivos. Por isso, você, investidor que fará a sua declaração de imposto

{'question': 'resuma o documento em anexo!',
 'chat_history': [HumanMessage(content='resuma o documento em anexo!'),
  AIMessage(content='O documento em anexo é um relatório anual consolidado de investimentos de uma pessoa chamada Carlos Eduardo Gevaerd Araujo para o ano de 2023. O relatório inclui a posição em ações de diversas empresas, como B3 S.A., Banco Bradesco, Engie Brasil, entre outras, com detalhes de quantidade, preço de fechamento e valor atualizado. Além disso, mostra a posição em CDB (Certificado de Depósito Bancário). Também são apresentados os proventos recebidos ao longo do ano, incluindo dividendos e juros sobre capital próprio de diferentes empresas. O total investido é de R$ 24.690,36. O relatório destaca a necessidade de solicitar informes de rendimentos à corretora para a declaração de imposto de renda.')],
 'answer': 'O documento em anexo é um relatório anual consolidado de investimentos de uma pessoa chamada Carlos Eduardo Gevaerd Araujo para o ano de 2023. O re